In [1]:
import tensorflow as tf
import numpy as np
import time
import math
import os
import cv2
import sys

## 可视化参数

In [2]:
#parameter summary 统计函数
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean) 
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)

In [3]:
'''
初始化权重函数
'''
def variable_with_weight_loss(shape, std, w1):
    var = tf.Variable(tf.truncated_normal(shape,stddev=std),dtype=tf.float32)
    
    if w1 is not None :
        weight_loss = tf.multiply(tf.nn.l2_loss(var),w1,name="weight_loss")
        tf.add_to_collection("losses",weight_loss)
    return var

'''
损失函数
'''
def loss_func(logits,labels):
    labels = tf.cast(labels,tf.int32)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                           labels=labels,name="cross_entropy_per_example")
    cross_entropy_mean = tf.reduce_mean(tf.reduce_sum(cross_entropy))
    tf.add_to_collection("losses",cross_entropy_mean)
    return tf.add_n(tf.get_collection("losses"),name="total_loss")



    
def forward(image_holder,train):
    
    #设计第一层卷积
    # 第一个卷积层
    with tf.name_scope('conv_layer1'):
        with tf.name_scope('W_conv1'):
            weight1 = variable_with_weight_loss(shape=[5,5,3,64],std=5e-2,w1=0)
            variable_summaries(weight1)
            kernel1 = tf.nn.conv2d(image_holder,weight1,[1,1,1,1],padding="SAME")
        with tf.name_scope('b_conv1'):
            bais1 = tf.Variable(tf.constant(0.0,dtype=tf.float32,shape=[64]))
            variable_summaries(bais1)
            conv1 = tf.nn.relu(tf.nn.bias_add(kernel1,bais1))
        with tf.name_scope('L1_pool'):
            pool1 = tf.nn.max_pool(conv1,[1,2,2,1],[1,2,2,1],padding="SAME")
        with tf.name_scope('L1_norml'):
            norm1 = tf.nn.lrn(pool1,4,bias=1.0,alpha=0.001 / 9,beta=0.75)

    #设计第二层卷积
    with tf.name_scope('conv_layer2'):
        with tf.name_scope('W_conv2'):
            weight2 = variable_with_weight_loss(shape=[5,5,64,64],std=5e-2,w1=0)
            variable_summaries(weight2)
            kernel2 = tf.nn.conv2d(norm1,weight2,[1,1,1,1],padding="SAME")
        with tf.name_scope('b_conv2'):
            bais2 = tf.Variable(tf.constant(0.1,dtype=tf.float32,shape=[64]))
            variable_summaries(bais2)
            conv2 = tf.nn.relu(tf.nn.bias_add(kernel2,bais2))
        with tf.name_scope('L2_pool'):
            norm2 = tf.nn.lrn(conv2,4,bias=1.0,alpha=0.01 / 9,beta=0.75)
        with tf.name_scope('L2_norml'):
            pool2 = tf.nn.max_pool(norm2,[1,3,3,1],[1,2,2,1],padding="SAME")

    #第一层全连接层
    # 把pool2从三维张量变为二维张量
    with tf.name_scope('fc_layer3'):
        with tf.name_scope('W_fc3'):
            pool_shape = pool2.get_shape().as_list()         # 得到pool2 输出矩阵的维度存入list中
            nodes = pool_shape[1] * pool_shape[2] * pool_shape[3]      # 提取特征的长度宽度深度 相乘得所有特征点的个数
            reshape = tf.reshape(pool2, [pool_shape[0], nodes])       # pool_shape[0]是一个batch的值
            #reshape = tf.reshape(pool2,[batch_size,-1])
            #dim = reshape.get_shape()[1].value
            weight3 = variable_with_weight_loss([nodes,384],std=0.04,w1=0.004)
            variable_summaries(weight3)
        with tf.name_scope('b_fc3'):
            bais3 = tf.Variable(tf.constant(0.1,shape=[384],dtype=tf.float32))
            variable_summaries(bais3)
            local3 = tf.nn.relu(tf.matmul(reshape,weight3)+bais3)
            #if train:
        # 如果是训练阶段，在这层使用0.5的dropout
        #local3 = tf.nn.dropout(local3, 0.2)
    #第二层全连接层
    with tf.name_scope('fc_layer4'):
        with tf.name_scope('W_fc4'):
            weight4 = variable_with_weight_loss([384,192],std=0.04,w1=0.004)
            variable_summaries(weight4)
        with tf.name_scope('b_fc4'):
            bais4 = tf.Variable(tf.constant(0.1,shape=[192],dtype=tf.float32))
            variable_summaries(bais4)
            local4 = tf.nn.relu(tf.matmul(local3,weight4)+bais4)

    #最后一层
    with tf.name_scope('fc_layer5'):
        with tf.name_scope('W_fc5'):
            weight5 = variable_with_weight_loss([192,10],std=1/192.0,w1=0)
            variable_summaries(weight5)
        with tf.name_scope('b_fc5'):
            bais5 = tf.Variable(tf.constant(0.0,shape=[10],dtype=tf.float32))
            variable_summaries(bais4)
            logits = tf.add(tf.matmul(local4,weight5),bais5)
    return logits

In [4]:


# 从tfrecord读出数据
def read_and_decode(filename):
    # 根据文件名生成一个队列
    filename_queue = tf.train.string_input_producer([filename])
    reader = tf.TFRecordReader()
    # 返回文件名和文件
    _, serialized_example = reader.read(filename_queue)   
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'image' : tf.FixedLenFeature([], tf.string),
                                           'label': tf.FixedLenFeature([], tf.int64),
                                       })
    # 获取图片数据
    images = tf.decode_raw(features['image'], tf.uint8)
    
    # tf.train.shuffle_batch必须确定shape
    images = tf.reshape(images, [24*24*3])
    
    # 图片预处理
    images = tf.cast(images, tf.float32) / 255.0
    images = tf.subtract(images, 0.5)
    images = tf.multiply(images, 2.0)
    # 获取label
    labels = tf.cast(features['label'], tf.int32)
    
    return images, labels


In [5]:
if __name__ == '__main__':

    #设置最大迭代次数
    max_steps = 20001
    #设置每次训练的数据大小
    batch_size = 100
    num_classes = 10
    epochs = 16
    buffer_size = 1000
    WIDTH = 24 
    HEIGHT = 24
    CHANNELS = 3
    BATCH_SIZE = 100
    LEARNING_RATE_DECAY = 0.96
    LEARNING_RATE_BASE = 0.001
    # tfrecord文件存放路径
    TFRECORD_FILE_TRAIN = 'train_images/train.tfrecords'
    TFRECORD_FILE_TEST = 'train_images/test.tfrecords'
    MODEL_SAVE_PATH = 'train_saver/digits'
    #获取数据增强后的训练集数据
    #images_train,labels_train = load_files(train_dir,num_classes,24*24*3)
    # 全局步数
    global_step = tf.Variable(0, trainable=False) 
    
 #获取数据增强后的训练集数据
    #images_train,labels_train = load_files(train_dir,num_classes,24*24*3)
    images_train,labels_train = read_and_decode(TFRECORD_FILE_TRAIN)
    image_batch, label_batch = tf.train.shuffle_batch(
        [images_train, labels_train], batch_size = BATCH_SIZE,
        capacity = 50000, min_after_dequeue=10000)
   
    #获取裁剪后的测试数据
    #images_test,labels_test = load_files(test_dir,num_classes,24*24*3)
    images_test,labels_test = read_and_decode(TFRECORD_FILE_TEST)
    images_batch_test, labels_batch_test = tf.train.shuffle_batch(
        [images_test, labels_test], batch_size = BATCH_SIZE,
        capacity = 50000, min_after_dequeue=10000)
        
    
    #定义模型的输入和输出数据
    with tf.name_scope("Input"):
        with tf.name_scope("image_hoder"):
            image_holder = tf.placeholder(dtype=tf.float32,shape=[batch_size,24*24*3])
            label_holder = tf.placeholder(dtype=tf.int32,shape=[batch_size])
            image_holder_reshape = tf.reshape(image_holder, [-1, WIDTH, HEIGHT, CHANNELS])
    with tf.name_scope("learning_rate"):
        # 学习率
        lr = tf.Variable(1e-3, dtype=tf.float32)
        tf.summary.scalar('learning_rate',lr)
        # 设置指数下降学习率
        lr_decay = tf.train.exponential_decay(
            LEARNING_RATE_BASE,
            global_step,
            BATCH_SIZE,
            LEARNING_RATE_DECAY,
            staircase=True) 
    
    

    logits = forward(image_holder_reshape,True)

    #设置优化算法使得成本最小
        # 定义优化器和训练op
    with tf.name_scope('optimizer'):
        with tf.name_scope('loss'):
            loss = loss_func(logits,label_holder)
            tf.summary.scalar('loss',loss)
    train_step = tf.train.AdamOptimizer(lr).minimize(loss,global_step=global_step)
    #获取最高类的分类准确率，取top1作为衡量标准
    #top_k_op = tf.nn.in_top_k(logits,label_holder,1)
    # 计算准确率
    # 把标签转成one_hot的形式
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            one_hot_labels = tf.one_hot(indices=tf.cast(label_holder, tf.int32), depth=num_classes)
            correct_prediction = tf.equal(tf.argmax(one_hot_labels,1),tf.argmax(logits,1))
        with tf.name_scope('accuracy'):    
            accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
            tf.summary.scalar('accuracy',accuracy)
#     #计算批次                           
#     batches_count_train = int(math.ceil(images_train.shape[0] / batch_size))
#     remainder_train = images_train.shape[0] % batch_size 
#     batches_count_test = int(math.ceil(images_test.shape[0] / batch_size))
#     remainder_test = images_test.shape[0] % batch_size
    

    #创建会话
    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()
    # 用于保存模型
    saver = tf.train.Saver()
    #开始训练
    # 图像增强队列
    tf.train.start_queue_runners()

    writer = tf.summary.FileWriter('logs/',sess.graph)
    
    #合并所有的summary
    merged = tf.summary.merge_all()
    ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
    for step in range(max_steps):
        start_time = time.time()
        #获取一个批次的数据和标签
        #images_batch,labels_batch = sess.run([images_train,labels_train])
        b_image, b_label = sess.run([image_batch, label_batch])
        _,loss_value,summary,step_g = sess.run([train_step,loss,merged,global_step],feed_dict={image_holder:b_image,
                                                                 label_holder:b_label})
        
        #获取计算时间
        duration = time.time() - start_time
        # 计算测试集准确率
        images_b,labels_b = sess.run([images_batch_test,labels_batch_test])
        #print("1",images_batch_test.shape)
        acc,loss_value_test = sess.run([accuracy,loss],feed_dict={image_holder:images_b,
                                                                 label_holder:labels_b}) 
        tf.summary.scalar('accuracy_test',acc)
        #计算每秒处理多少张图片
        per_images_second = batch_size / duration
        #获取时间
        sec_per_batch = float(duration)
        # 降低学习率
        lr_decay_val = sess.run(lr_decay) 
        sess.run(tf.assign(lr, lr_decay_val))     
        learning_rate = sess.run(lr)    
        sys.stdout.write('\r>> Step:%d,  duration:%.3f,  per_images_second:%.2f,  loss:%.3f,  accuracy:%.3f,  test loss:%.3f,  learning_rate:%.5f' % 
                         (step_g,duration,per_images_second,loss_value,acc,loss_value_test,learning_rate))
        sys.stdout.flush() 
        writer.add_summary(summary,step)
        if step_g % 500 == 0:
            # 训练完成后保存训练模型
            saver.save(sess, "./train_saver/digits/crack_captcha.model", global_step=global_step, meta_graph_suffix='meta', write_meta_graph=True) 
            tf.train.write_graph(sess.graph_def, "./train_saver/digits", "crack_captcha.pb", False)
            
        

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Queue-bas

KeyboardInterrupt: 